In [1]:
import os 
import sys 
import zipfile

from skimage.io import imread

import numpy as np
import pandas as pd
import tensorflow as tf

import numpy as np
def scale_image(image1):
    im = image1.astype(np.float32)
    im /= 255
    return im

print(os.getcwd())

C:\Users\e7zv5qo\Jupyter_Notebooks\UCB_Deep_Learning_03\Module3


In [2]:
train_labels = pd.read_csv("./data/train_labels.csv")
print(train_labels)
train_images = list(train_labels['id'])
train_images = [ name + '.tif' for name in train_images]
labels = list(train_labels['label'])
print(train_images[0:5])

                                              id  label
0       f38a6374c348f90b587e046aac6079959adf3835      0
1       c18f2d887b7ae4f6742ee445113fa1aef383ed77      1
2       755db6279dae599ebb4d39a9123cce439965282d      0
3       bc3f0c64fb968ff4a8bd33af6971ecae77c75e08      0
4       068aba587a4950175d04c680d38943fd488d6a9d      0
...                                          ...    ...
220020  53e9aa9d46e720bf3c6a7528d1fca3ba6e2e49f6      0
220021  d4b854fe38b07fe2831ad73892b3cec877689576      1
220022  3d046cead1a2a5cbe00b2b4847cfb7ba7cf5fe75      0
220023  f129691c13433f66e1e0671ff1fe80944816f5a2      0
220024  a81f84895ddcd522302ddf34be02eb1b3e5af1cb      1

[220025 rows x 2 columns]
['f38a6374c348f90b587e046aac6079959adf3835.tif', 'c18f2d887b7ae4f6742ee445113fa1aef383ed77.tif', '755db6279dae599ebb4d39a9123cce439965282d.tif', 'bc3f0c64fb968ff4a8bd33af6971ecae77c75e08.tif', '068aba587a4950175d04c680d38943fd488d6a9d.tif']


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_images, labels, random_state=0, train_size = 210000)
print(len(X_train))
print(len(y_train))
print(len(X_valid))
print(len(y_valid))
print(y_train[0:10])
print(np.mean(labels))

210000
210000
10025
10025
[1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
0.40503124644926713


In [4]:
def read_image_collection(file_list, file_path):
    return np.array([
        scale_image(imread(file_path+file_name))
        for file_name in file_list])

In [5]:
print(X_train[0:5])

['b39869928d6bcdc4c3be03ac34b781186814a929.tif', 'c667053b8177533b69710ac12671a494f3446e74.tif', 'a265cdfc83c4c961bc4186638a3564dd5bb86c1c.tif', '8eb04aba813097c3abd141016eb6d3fd065dcd7a.tif', '1013bc0def9e87f75af3a2f9bfa6e3a2ccb7f732.tif']


In [8]:
X_train_images = read_image_collection(X_train, './data/train/')
print(X_train_images.shape)
X_valid_images = read_image_collection(X_valid, './data/train/')
print(X_valid_images.shape)
print('Ready')

(210000, 96, 96, 3)
(10025, 96, 96, 3)
Ready


In [9]:
gbytes = 210000 * 96 * 96 * 3 * 4/ 10**9
print(gbytes)

23.22432


In [48]:
from tensorflow import keras

def modelArch4():
    model = keras.models.Sequential([
        keras.layers.Conv2D(32, 3, activation='relu', padding='valid',
                           input_shape=[96,96,3]),
        keras.layers.MaxPool2D(2),
        keras.layers.Conv2D(48, 3, activation=tf.nn.relu, padding='valid'),
        keras.layers.MaxPool2D(2),
        keras.layers.Conv2D(64, 3, activation=tf.nn.relu, padding='valid'),
        keras.layers.MaxPool2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation=tf.nn.relu),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(32,  activation=tf.nn.relu),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation=tf.nn.sigmoid),
    ])
    return model

model = modelArch4()
model.summary()
opt = keras.optimizers.Adam(learning_rate=0.00020)
loss = keras.losses.BinaryCrossentropy()
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 94, 94, 32)        896       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 45, 45, 48)        13872     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 22, 22, 48)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 20, 20, 64)        27712     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 6400)             

In [49]:
history = model.fit(x=X_train_images,y=np.array(y_train),verbose=1,validation_data=(X_valid_images,np.array(y_valid)),epochs=2)

Epoch 1/2
6563/6563 [==============================] - 807s 123ms/step - loss: 0.4683 - accuracy: 0.7910 - val_loss: 0.3609 - val_accuracy: 0.8468
Epoch 2/2
6563/6563 [==============================] - 788s 120ms/step - loss: 0.3637 - accuracy: 0.8496 - val_loss: 0.3084 - val_accuracy: 0.8717


In [51]:
history = model.fit(x=X_train_images,y=np.array(y_train),verbose=1,validation_data=(X_valid_images,np.array(y_valid)),epochs=1)

6563/6563 [==============================] - 794s 121ms/step - loss: 0.3120 - accuracy: 0.8750 - val_loss: 0.2565 - val_accuracy: 0.8934


In [53]:
history = model.fit(x=X_train_images,y=np.array(y_train),verbose=1,validation_data=(X_valid_images,np.array(y_valid)),epochs=1)

6563/6563 [==============================] - 818s 125ms/step - loss: 0.2767 - accuracy: 0.8916 - val_loss: 0.2659 - val_accuracy: 0.8897


In [ ]:
history = model.fit(x=X_train_images,y=np.array(y_train),verbose=1,validation_data=(X_valid_images,np.array(y_valid)),epochs=1)

In [15]:
sample = pd.read_csv('./data/sample_submission.csv')
sample_names = list(sample['id'])
sample_names[0:20]
## train_images = [ name + '.tif' for name in train_images]
test_images = [name + '.tif' for name in sample_names]
test_images[0:10]

['0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5.tif',
 '95596b92e5066c5c52466c90b69ff089b39f2737.tif',
 '248e6738860e2ebcf6258cdc1f32f299e0c76914.tif',
 '2c35657e312966e9294eac6841726ff3a748febf.tif',
 '145782eb7caa1c516acbe2eda34d9a3f31c41fd6.tif',
 '725dabe6ecccc68b958a2c7dd75bcbf362c7cb03.tif',
 'aa0307865281d4484ddf8c637c348292968b93a7.tif',
 'f4e5dc9c949920f1b3362982e15e99bf6f3ef83b.tif',
 '95e08c9cedc28a9b4a86f4fc1e06c1972134be08.tif',
 'ee1f5345a8d4e28403b7e61b97b5f76c201ce9cb.tif']

In [16]:
X_test_images = read_image_collection(test_images, './data/test/')
print(X_test_images.shape)

(57458, 96, 96, 3)


In [29]:
print(X_test_images.shape)

(57458, 96, 96, 3)


In [54]:
## predictions = (model.predict(X_test_images) > 0.5).astype("int32")
predict_proba = model.predict(X_test_images)
## predictions[0:10]
print(predict_proba[0:10])
predictions = (predict_proba > 0.5).astype("int32")
print(predictions[0:10])
print(sum(predictions))
print(len(predictions))
df = pd.DataFrame(sample_names)
df.columns= ['id']
df['label'] = predictions
df
df.to_csv('./data/submission14.csv',index=False)
print('Submission 14')

[[0.01387101]
 [0.728766  ]
 [0.00339532]
 [0.13174924]
 [0.02223381]
 [0.7158254 ]
 [0.9858694 ]
 [0.8292243 ]
 [0.38539946]
 [0.64895415]]
[[0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]]
[23456]
57458
Submission 14
